In [1]:
import keras
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score
from keras import models
from keras.wrappers.scikit_learn import KerasRegressor
from keras.models import Sequential,model_from_json
from keras.layers import Dense, Dropout, LSTM, Activation
%matplotlib inline
from decimal import Decimal
import warnings
warnings.filterwarnings("ignore")
import pickle
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,Normalizer, MinMaxScaler
from sklearn.cross_validation import cross_val_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb

Using TensorFlow backend.


In [2]:
df=pd.read_csv('C:\\Users\\lengada1\\NCSU\\DC_ten_skus.csv')
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.weekday_name
day_dummy=pd.get_dummies(df.Day)
df=pd.concat([df,day_dummy],axis=1)
df.drop(['Day','Date'],inplace=True,axis=1)

y=df['Sales']

In [3]:
df.head()
df.shape

(942, 16)

In [4]:
X=df.drop(['id','DayOfWeek','Customers','Sales'],inplace=False,axis=1)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,10,2013,1,0,0,0,0,0,1,0
1,10,0,10,2013,1,0,0,0,0,0,0,1
2,10,0,8,2013,1,0,0,0,0,1,0,0
3,10,0,8,2013,1,1,0,0,0,0,0,0
4,10,0,1,2013,1,0,0,1,0,0,0,0


In [5]:
for obs in range(1,8):
    X["Sales_T"+str(obs)]=df['Sales'].shift(obs)


In [9]:
X['Mov_avg']=df['Sales'].rolling( window=7).mean().shift(1) 
y=y[7:]
X=X[7:]
y.reset_index(drop=True, inplace=True)
X.reset_index(drop=True, inplace=True)
X.head()

,Open,Promo,SchoolHoliday,Year,Month,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,Sales_T1,Sales_T2,Sales_T3,Sales_T4,Sales_T5,Sales_T6,Sales_T7,Mov_avg
0,10,0,0,2013,1,0,0,0,0,0,1,0,50984.0,0.0,43300.0,65006.0,64112.0,64297.0,72552.0,43144.000000
1,10,0,0,2013,1,0,0,0,0,0,0,1,48246.0,50984.0,0.0,43300.0,65006.0,64112.0,64297.0,53508.571429
2,10,0,0,2013,1,0,0,0,0,1,0,0,45400.0,48246.0,50984.0,0.0,43300.0,65006.0,64112.0,54060.142857
3,10,0,0,2013,1,1,0,0,0,0,0,0,46978.0,45400.0,48246.0,50984.0,0.0,43300.0,65006.0,55669.285714
4,10,0,0,2013,1,0,0,1,0,0,0,0,51511.0,46978.0,45400.0,48246.0,50984.0,0.0,43300.0,57012.142857


In [10]:
std= preprocessing.StandardScaler().fit(X)
X = std.transform(X)
X

array([[ 0.45493259, -0.7871126 , -0.60383607, ...,  0.446326  ,
         0.75093448, -1.17714063],
       [ 0.45493259, -0.7871126 , -0.60383607, ...,  0.43947033,
         0.44510427,  0.16027228],
       [ 0.45493259, -0.7871126 , -0.60383607, ...,  0.47259988,
         0.43825041,  0.23144539],
       ...,
       [ 0.45493259,  1.27046626,  2.69046067, ...,  0.16075963,
         0.06206629,  0.2401646 ],
       [ 0.45493259,  1.27046626,  2.69046067, ...,  0.0764905 ,
         0.15961334, -0.116936  ],
       [ 0.45493259,  1.27046626,  2.69046067, ..., -0.27237037,
         0.07536648, -0.41716817]])

In [29]:
cv=3

RF=RandomForestRegressor( n_estimators=1000,max_depth=10,max_features=14);
rf_models=[];
rf_models=(cross_val_score(RF,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )


GB=ensemble.GradientBoostingRegressor(n_estimators= 1000,max_depth= 8, max_features=14,
          learning_rate= 0.01, loss= 'ls');
gb_models=[];
gb_models=(cross_val_score(GB,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

XG=xgb.XGBRegressor(n_estimators= 900,learning_rate =0.01, max_depth=6,gamma=2)
xg_models=[]
xg_models=(cross_val_score(XG,X,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

In [30]:
def create_MLP():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP = KerasRegressor(build_fn=create_MLP,
                               epochs=300,
                               batch_size=20,
                               verbose=0)

mlp_models=[]
mlp_models=(cross_val_score(MLP,X,y,cv=cv).mean().round(0)  )

In [37]:
mae=-pd.DataFrame(np.column_stack((mlp_models,rf_models,gb_models,xg_models)) )
mae=mae.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
best=mae.idxmin(axis=1)
best.astype('category')
mae["Best"]=best
mae

,MLP,RF,GB,XG,Best
0,4328.0,3462.0,3288.0,3286.0,XG


In [32]:
#Same models as above, but configured here to predict over entire X (935 days)
def RF_final(X, y):
    tree=RF
    tree.fit(X,y)
    pred=tree.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'RF'})
    return pred

def MLP_final(X,y):
    model = MLP
    model.fit(X,y)
    pred=model.predict(X)
    pred=pd.DataFrame(pred)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'MLP'})
    return pred

def GB_final(X,y):
    model = GB
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'GB'})
    return pred

def XG_final(X,y):
    model=XG
    model.fit(X,y)
    prediction=model.predict(X)
    pred=pd.DataFrame(prediction)
    pred.reset_index(drop=True, inplace=True)
    pred=pred.rename(columns = {0:'XG'})
    return pred

In [50]:
#Using aggregated SKU sales data, the best performing model is selected based on K-fold. 
#Then best model is used to predict full data set 

bm_dict={}
for sku in range(0,1):        
        if  mae["Best"][0]=="RF":
            bm_dict[sku]=RF_final(X,y);
                    
        elif mae["Best"][0]=="NN":
            bm_dict[sku]=MLP_final(X,y);
                                
        elif mae["Best"][0]=="GB":
            bm_dict[sku]=GB_final(X,y); 
                        
        elif mae["Best"][0]=="XG":
            bm_dict[sku]=XG_final(X,y);

In [51]:
# Ceate Phase 2 data set which is comprised of both aggregate SKU data and forecasts
dc_pred=pd.DataFrame(bm_dict[0])
sku_pred=pd.read_csv('C:\\Users\\lengada1\\NCSU\\prediction_skus.csv')
sku_pred=sku_pred.iloc[:,1:]
X2=pd.concat([sku_pred,dc_pred,pd.DataFrame(X)],axis=1)
X2.head()

,1,2,3,4,5,6,7,8,9,10,...,10,11,12,13,14,15,16,17,18,19
0,4028.070155,3933.607419,5174.478232,7432.9385,3725.016630,5607.321375,6806.916205,4207.681667,4355.2990,4575.576329,...,2.444881,-0.409018,-0.049789,-1.930105,-0.329533,0.472984,0.441196,0.446326,0.750934,-1.177141
1,3939.086922,4632.660739,5142.490788,7006.6010,3997.977265,5275.288788,6387.782391,3888.736040,3897.4960,4557.150310,...,-0.409018,2.444881,-0.151100,-0.046784,-1.929420,-0.329230,0.474263,0.439470,0.445104,0.160272
2,3890.810664,3840.314749,5054.772569,7085.2993,3719.835842,5551.322346,6798.710602,4839.750751,4251.4766,4520.816120,...,-0.409018,-0.409018,-0.256407,-0.147924,-0.045618,-1.929519,-0.328579,0.472600,0.438250,0.231445
3,4253.015695,3886.589696,5778.068833,7839.3604,4058.536214,5698.072570,7586.416943,3999.500579,5356.4190,4761.602488,...,-0.409018,-0.409018,-0.198018,-0.253054,-0.146784,-0.045243,-1.930121,-0.331774,0.471371,0.439084
4,4986.052622,2519.789920,4131.315951,9462.2980,1784.665517,3649.817545,4877.134636,2423.252425,5014.5513,4516.572428,...,-0.409018,-0.409018,-0.030289,-0.194763,-0.251940,-0.146435,-0.044370,-1.936371,-0.332790,0.612363


In [52]:
rng=range(0,33)
new_cols =  ['' + str(i) for i in rng]
X2.columns = new_cols[:32]
std2= preprocessing.StandardScaler().fit(X2)
X2 = std2.transform(X2)

In [53]:
def create_MLP2():
    mlp=models.Sequential()
    mlp.add(Dense(20, input_dim=X2.shape[1], activation='relu'))
    #mlp.add(Dense(20, activation='relu'))
    mlp.add(Dense(30, activation='relu'))
    mlp.add(Dense(10, activation='relu'))    
    mlp.add(Dense(1, activation='relu'))
    mlp.compile(loss='mae', optimizer='adam', metrics=['mae'])
    return mlp

MLP2 = KerasRegressor(build_fn=create_MLP2,
                               epochs=300,
                               batch_size=20,
                               verbose=0)

In [54]:
rf2_models=[];
rf2_models=(cross_val_score(RF,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0) )

gb2_models=[];
gb2_models=(cross_val_score(GB,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )
   

xg2_models=[];
xg2_models=(cross_val_score(XG,X2,y,cv=cv, scoring ='mean_absolute_error').mean().round(0)  )

mlp2_models=[];
mlp2_models=(cross_val_score(MLP2,X2,y,cv=cv, scoring='mean_absolute_error').mean().round(0)  )

In [63]:
mae2=-pd.DataFrame(np.column_stack((mlp2_models,rf2_models,gb2_models,xg2_models)) )
mae2=mae2.rename(columns = {0:'MLP',1:'RF',2:'GB',3:'XG'})
mae2

,MLP,RF,GB,XG
0,1408.0,1212.0,1169.0,1064.0


In [64]:
#Using forecasted data and no empirical sales data, the best performaning model is selected based on Train/Test. 
## NOTE this improvement is not the true delta.  Only a final forecast between Stage 1 vs 2 will tell. 

best2=mae2.idxmin(axis=1)
best2.astype('category')
mae2["Best"]=best2
both_mae=pd.concat([mae,mae2],axis=0)
both_mae.reset_index(drop=True, inplace=True)

# Phase 1 & 2 DC Models compared based on Test set MAE scores. 
both_mae['min']=both_mae.min(axis=1)
both_mae['error reduction']= ((both_mae['min'][0]-both_mae['min'][1])/both_mae['min'][0] ).round(3)
both_mae

,MLP,RF,GB,XG,Best,min,error reduction
0,4328.0,3462.0,3288.0,3286.0,XG,3286.0,0.676
1,1408.0,1212.0,1169.0,1064.0,XG,1064.0,0.676


In [57]:
bm2_dict={}
for sku in range(0,1):
        if  mae2["Best"][0]=="RF":
            bm2_dict[sku]=RF_final(X2,y);
                    
        elif mae2["Best"][0]=="NN":
            bm2_dict[sku]=MLP_final(X2,y);
                                
        elif mae2["Best"][0]=="GB":
            bm2_dict[sku]=GB_final(X2,y); 
                        
        elif mae2["Best"][0]=="XG":
            bm2_dict[sku]=XG_final(X2,y);    

In [59]:
tier_pred=pd.concat([bm_dict[0],bm2_dict[0],y],axis=1)
tier_pred.columns = ['DC_Phase1', 'DC_Phase2',"Sales"]
tier_pred.to_csv('C:\\Users\\lengada1\\NCSU\\DC_tier_predictions.csv')
tier_pred.round()

,DC_Phase1,DC_Phase2,Sales
0,48004.0,48240.0,48246
1,47565.0,46644.0,45400
2,47836.0,47146.0,46978
3,51752.0,51472.0,51511
4,43841.0,42870.0,42506
5,-17.0,13.0,0
6,73451.0,72135.0,71624
7,67019.0,67874.0,68454
8,65622.0,66122.0,66359
9,62041.0,61317.0,60730


In [60]:
# END NOTEBOOK

In [61]:
# The rest of this notebook is just a reference and aid to allow collaborators to save models and retrive and apply them without retraining. 
# The file names are not correct, thus just a reference. 

# also look at joblib.  https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
model_json = model.to_json()
with open("NCSU_DC.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("NCSU_DC.h5")
print("Saved model to disk")

NameError: name 'model' is not defined

In [ ]:
# load json and create model
json_file = open('NCSU_DC.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("NCSU_DC.h5")
print("Loaded model from disk")

In [ ]:
# evaluate loaded model on test data
loaded_model.compile(loss='mse', optimizer='adam', metrics=['mae','mse'])
score = loaded_model.evaluate(Xs, y, verbose=0)
print("%s: %.0f"%  (loaded_model.metrics_names[1], score[1]))
print("%s: %.1f"%  (loaded_model.metrics_names[2], score[2]/10000000))